## 1. Libraries

In [185]:
import requests
import json
import pandas as pd
import datetime
import time
import random
import numpy as np
import re
import datetime as dt # Ojo con esto
import pytz
import os
import matplotlib.pyplot as plt

In [186]:
FechaHoy = pd.Timestamp.today().date().strftime('%Y-%m-%d')# %H:%M')

Now = dt.datetime.now(pytz.timezone('America/Bogota')).strftime('%b-%d-%Y %H:%M')

In [187]:
FechaHoy_aniomes = pd.Timestamp.today().date().strftime('%Y%m')# %H:%M')

In [188]:
d = {'last_update':[Now]}

## 2. Process

### 2.1 Colombian official exchange rate (COP/USD)

#### The source of this data is the Colombian Central Bank through Colombian Government Open Data platform

In [189]:
df_trm = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
trm_vigente = df_trm.loc[0][0]
trm_vigente

4092.33

In [190]:
trm_vig = {'TRM':[trm_vigente]}
TRM_Vigente = pd.DataFrame(data=trm_vig)
TRM_Vigente

,TRM
0,4092.33


### 2.2 Getting data from yfinance API

In [191]:
#pip install yahoo_fin

In [192]:
#!pip install yfinance --upgrade --no-cache-dir

In [193]:
import yfinance as yf

In [194]:
msft = yf.Ticker("MSFT")
print(msft.info)

{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and pow

In [195]:
df = pd.DataFrame([msft.info], columns=msft.info.keys())
df

,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,...,returnOnEquity,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio
0,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,software-infrastructure,Software - Infrastructure,...,0.39107,50418876416,94966996992,0.272,0.128,0.69441,0.49714,0.47587,USD,2.2866


In [196]:
wti_oil = yf.Ticker("CL=F") #WTI
print(wti_oil.info)

{'maxAge': 86400, 'priceHint': 2, 'previousClose': 77.1, 'open': 76.79, 'dayLow': 75.3, 'dayHigh': 76.87, 'regularMarketPreviousClose': 77.1, 'regularMarketOpen': 76.79, 'regularMarketDayLow': 75.3, 'regularMarketDayHigh': 76.87, 'volume': 166379, 'regularMarketVolume': 166379, 'averageVolume': 324920, 'averageVolume10days': 258977, 'averageDailyVolume10Day': 258977, 'bid': 76.33, 'ask': 76.34, 'bidSize': 900, 'askSize': 1100, 'expireDate': 1702944000, 'openInterest': 343803, 'fiftyTwoWeekLow': 63.64, 'fiftyTwoWeekHigh': 95.03, 'fiftyDayAverage': 84.0108, 'twoHundredDayAverage': 78.1646, 'currency': 'USD', 'exchange': 'NYM', 'quoteType': 'FUTURE', 'symbol': 'CLF24.NYM', 'underlyingSymbol': 'CL=F', 'shortName': 'Crude Oil Jan 24', 'firstTradeDateEpochUtc': 967003200, 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EST', 'uuid': '2e63caf7-96c3-3195-95bd-9fdc98ac5dd3', 'gmtOffSetMilliseconds': -18000000, 'trailingPegRatio': None}


In [197]:
df_wti = pd.DataFrame([wti_oil.info], columns=wti_oil.info.keys())
df_wti = df_wti.filter(['open'])
df_wti

,open
0,76.79


In [198]:
df_wti.rename(columns = {'open':'WTI Last Open'}, inplace = True)
df_wti

,WTI Last Open
0,76.79


In [199]:
brent_oil = yf.Ticker("BZ=F") #BRENT
print(brent_oil.info)

{'maxAge': 86400, 'priceHint': 2, 'previousClose': 81.96, 'open': 81.63, 'dayLow': 80.18, 'dayHigh': 81.86, 'regularMarketPreviousClose': 81.96, 'regularMarketOpen': 81.63, 'regularMarketDayLow': 80.18, 'regularMarketDayHigh': 81.86, 'volume': 10702, 'regularMarketVolume': 10702, 'averageVolume': 32092, 'averageVolume10days': 30189, 'averageDailyVolume10Day': 30189, 'bid': 81.4, 'ask': 81.42, 'askSize': 1200, 'expireDate': 1701388800, 'openInterest': 29095, 'fiftyTwoWeekLow': 68.2, 'fiftyTwoWeekHigh': 97.63, 'fiftyDayAverage': 87.7044, 'twoHundredDayAverage': 82.51165, 'currency': 'USD', 'exchange': 'NYM', 'quoteType': 'FUTURE', 'symbol': 'BZF24.NYM', 'underlyingSymbol': 'BZ=F', 'shortName': 'Brent Crude Oil Last Day Financ', 'firstTradeDateEpochUtc': 1185768000, 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EST', 'uuid': '06672b05-645b-3b9d-9410-aed8c09764c9', 'gmtOffSetMilliseconds': -18000000, 'trailingPegRatio': None}


In [200]:
df_brent_oil = pd.DataFrame([brent_oil.info], columns=brent_oil.info.keys())
df_brent_oil = df_brent_oil.filter(['open'])
df_brent_oil

,open
0,81.63


In [201]:
df_brent_oil.rename(columns = {'open':'Brent Last Open'}, inplace = True)
df_brent_oil

,Brent Last Open
0,81.63


In [202]:
gold = yf.Ticker("GC=F") #Gold
print(gold.info)

{'maxAge': 86400, 'priceHint': 2, 'previousClose': 1992.8, 'open': 1991.2, 'dayLow': 1990.9, 'dayHigh': 2003.9, 'regularMarketPreviousClose': 1992.8, 'regularMarketOpen': 1991.2, 'regularMarketDayLow': 1990.9, 'regularMarketDayHigh': 2003.9, 'volume': 171304, 'regularMarketVolume': 171304, 'averageVolume': 697, 'averageVolume10days': 1299, 'averageDailyVolume10Day': 1299, 'bid': 2002.1, 'ask': 2002.2, 'bidSize': 1400, 'askSize': 800, 'expireDate': 1703635200, 'openInterest': 154625, 'fiftyTwoWeekLow': 1737.9, 'fiftyTwoWeekHigh': 2072.0, 'fiftyDayAverage': 1933.248, 'twoHundredDayAverage': 1937.846, 'currency': 'USD', 'exchange': 'CMX', 'quoteType': 'FUTURE', 'symbol': 'GCZ23.CMX', 'underlyingSymbol': 'GC=F', 'shortName': 'Gold Dec 23', 'firstTradeDateEpochUtc': 967608000, 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EST', 'uuid': '14f8f2ec-b820-38f7-ba5f-b3f5a47aec05', 'gmtOffSetMilliseconds': -18000000, 'trailingPegRatio': None}


In [203]:
df_gold = pd.DataFrame([gold.info], columns=gold.info.keys())
df_gold = df_gold.filter(['open'])
df_gold

,open
0,1991.2


In [204]:
df_gold.rename(columns = {'open':'Gold Last Open'}, inplace = True)
df_gold

,Gold Last Open
0,1991.2


In [205]:
# Available data

sorted(df.columns)

['52WeekChange',
 'SandP52WeekChange',
 'address1',
 'ask',
 'askSize',
 'auditRisk',
 'averageDailyVolume10Day',
 'averageVolume',
 'averageVolume10days',
 'beta',
 'bid',
 'bidSize',
 'boardRisk',
 'bookValue',
 'city',
 'companyOfficers',
 'compensationAsOfEpochDate',
 'compensationRisk',
 'country',
 'currency',
 'currentPrice',
 'currentRatio',
 'dateShortInterest',
 'dayHigh',
 'dayLow',
 'debtToEquity',
 'dividendRate',
 'dividendYield',
 'earningsGrowth',
 'earningsQuarterlyGrowth',
 'ebitda',
 'ebitdaMargins',
 'enterpriseToEbitda',
 'enterpriseToRevenue',
 'enterpriseValue',
 'exDividendDate',
 'exchange',
 'fiftyDayAverage',
 'fiftyTwoWeekHigh',
 'fiftyTwoWeekLow',
 'financialCurrency',
 'firstTradeDateEpochUtc',
 'fiveYearAvgDividendYield',
 'floatShares',
 'forwardEps',
 'forwardPE',
 'freeCashflow',
 'fullTimeEmployees',
 'gmtOffSetMilliseconds',
 'governanceEpochDate',
 'grossMargins',
 'heldPercentInsiders',
 'heldPercentInstitutions',
 'impliedSharesOutstanding',
 'ind

In [206]:
tickers = ['AAPL','MSFT','GOOGL','AMZN',"META","TSLA",'LVMHF','JPM','WMT','PG','KO','PEP','BAC','TM','CRM',
          'NFLX','DIS','INTC','AXP','SBUX','BUD','PBR','C','ABNB','BMWYY','MELI','SAN','CL','TGT','HEINY',
          'BBVA','BASFY','KMB','NU','BK','KR','SPOT','EC','MT','TS','CLX','BG','BAM','CX','ZLNDY',
          'TX','WHR','GLOB','CIB','TGLS','BCS','BMO','BNS','BYND','CABGY','CLVR','COST','DASH','DLVHF','DOCU',
           'EXXRF','GM','GS','HCMLY','F',
           #'INDEXF',
           'LIN','LYFT','MA','NKE','NSANY','PFE','PLTR','QSR','RY','SNYNF',
           'UBER','USB','VIVEF','WFC','AUR','NVDA','PATH','BNR.DE','AON','SNOW','NOW','WDAY','TWLO','RIVN','LCID'
           #,'WORK','EXTO-WI'
           ]

In [207]:
tickers_stats = {}

In [208]:
for ticker in tickers:
    temp = yf.Ticker(ticker)
    temp2 = temp.info
    #temp = temp.iloc[:,:2]
    #temp2.columns = ["Attribute", "Recent"]
    tickers_stats[ticker] = temp2
#tickers_stats

In [209]:
df_info = pd.DataFrame.from_dict(tickers_stats)

In [210]:
rows = ['symbol','shortName','beta','open','previousClose','ebitda','ebitdaMargins','enterpriseToEbitda','marketCap','enterpriseValue','pegRatio','priceToBook','totalDebt','totalRevenue','industry',
                 'sector','industryDisp','fullTimeEmployees']
df_info2 = df_info.loc[rows]

In [211]:
df_info3 = df_info2.transpose()

In [212]:
df_info4 = df_info3.reset_index()

In [213]:
df_info5 = df_info4.drop(columns=['index'])

In [214]:
df_info6 = df_info5.sort_values(by=['marketCap'], ascending=False)

In [215]:
df_info6

,symbol,shortName,beta,open,previousClose,ebitda,ebitdaMargins,enterpriseToEbitda,marketCap,enterpriseValue,pegRatio,priceToBook,totalDebt,totalRevenue,industry,sector,industryDisp,fullTimeEmployees
0,AAPL,Apple Inc.,1.305,191.49,191.31,125820002304,0.32827,24.144,2951143751680,3037771857920,4.74,47.473106,123930001408,383285002240,Consumer Electronics,Technology,Consumer Electronics,161000
1,MSFT,Microsoft Corporation,0.885,377.33,377.85,108530999296,0.49714,25.523,2803560087552,2770016403456,2.18,12.699986,105681002496,218310000640,Software - Infrastructure,Technology,Software - Infrastructure,221000
2,GOOGL,Alphabet Inc.,1.05,138.03,138.49,96071000064,0.32333,17.1,1725866180608,1642851860480,1.24,6.297911,29446000640,297131999232,Internet Content & Information,Communication Services,Internet Content & Information,182381
3,AMZN,"Amazon.com, Inc.",1.175,144.57,146.71,73978003456,0.13353,21.871,1507730522112,1618001657856,0.63,8.236888,166064996352,554027974656,Internet Retail,Consumer Cyclical,Internet Retail,1500000
80,NVDA,NVIDIA Corporation,1.694,498.52,487.16,22161000448,0.49389,53.97,1191900741632,1196031213568,0.36,35.77366,11027000320,44870000640,Semiconductors,Technology,Semiconductors,26196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LYFT,"Lyft, Inc.",1.789,10.27,10.29,-784977984,-0.18029,-4.456,4004568064,3497931008,-0.28,8.548657,1120690944,4353995776,Software - Application,Technology,Software - Application,4419
79,AUR,"Aurora Innovation, Inc.",2.669,2.13,2.16,-815000000,0.0,-2.376,3305543168,1936743808,NaN,1.542553,125000000,2000000,Information Technology Services,Technology,Information Technology Services,1700
49,TGLS,Tecnoglass Inc.,1.814,34.3,34.45,312592992,0.36785,5.357,1634437632,1674482176,9.23,3.181636,181148992,849779968,Building Materials,Basic Materials,Building Materials,8770
53,BYND,"Beyond Meat, Inc.",2.249,6.6,6.6,-217475008,-0.62201,-6.552,435651072,1424972928,-0.18,NaN,1216547968,349635008,Packaged Foods,Consumer Defensive,Packaged Foods,787


In [216]:
df_info7 = df_info6.copy()

In [217]:
df_info7

,symbol,shortName,beta,open,previousClose,ebitda,ebitdaMargins,enterpriseToEbitda,marketCap,enterpriseValue,pegRatio,priceToBook,totalDebt,totalRevenue,industry,sector,industryDisp,fullTimeEmployees
0,AAPL,Apple Inc.,1.305,191.49,191.31,125820002304,0.32827,24.144,2951143751680,3037771857920,4.74,47.473106,123930001408,383285002240,Consumer Electronics,Technology,Consumer Electronics,161000
1,MSFT,Microsoft Corporation,0.885,377.33,377.85,108530999296,0.49714,25.523,2803560087552,2770016403456,2.18,12.699986,105681002496,218310000640,Software - Infrastructure,Technology,Software - Infrastructure,221000
2,GOOGL,Alphabet Inc.,1.05,138.03,138.49,96071000064,0.32333,17.1,1725866180608,1642851860480,1.24,6.297911,29446000640,297131999232,Internet Content & Information,Communication Services,Internet Content & Information,182381
3,AMZN,"Amazon.com, Inc.",1.175,144.57,146.71,73978003456,0.13353,21.871,1507730522112,1618001657856,0.63,8.236888,166064996352,554027974656,Internet Retail,Consumer Cyclical,Internet Retail,1500000
80,NVDA,NVIDIA Corporation,1.694,498.52,487.16,22161000448,0.49389,53.97,1191900741632,1196031213568,0.36,35.77366,11027000320,44870000640,Semiconductors,Technology,Semiconductors,26196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LYFT,"Lyft, Inc.",1.789,10.27,10.29,-784977984,-0.18029,-4.456,4004568064,3497931008,-0.28,8.548657,1120690944,4353995776,Software - Application,Technology,Software - Application,4419
79,AUR,"Aurora Innovation, Inc.",2.669,2.13,2.16,-815000000,0.0,-2.376,3305543168,1936743808,NaN,1.542553,125000000,2000000,Information Technology Services,Technology,Information Technology Services,1700
49,TGLS,Tecnoglass Inc.,1.814,34.3,34.45,312592992,0.36785,5.357,1634437632,1674482176,9.23,3.181636,181148992,849779968,Building Materials,Basic Materials,Building Materials,8770
53,BYND,"Beyond Meat, Inc.",2.249,6.6,6.6,-217475008,-0.62201,-6.552,435651072,1424972928,-0.18,NaN,1216547968,349635008,Packaged Foods,Consumer Defensive,Packaged Foods,787


In [218]:
# Applying format to the data

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'B', 'T', 'P'][magnitude])

In [219]:
df_info7['marketCap2'] =df_info7['marketCap'].apply(human_format)
df_info7

,symbol,shortName,beta,open,previousClose,ebitda,ebitdaMargins,enterpriseToEbitda,marketCap,enterpriseValue,pegRatio,priceToBook,totalDebt,totalRevenue,industry,sector,industryDisp,fullTimeEmployees,marketCap2
0,AAPL,Apple Inc.,1.305,191.49,191.31,125820002304,0.32827,24.144,2951143751680,3037771857920,4.74,47.473106,123930001408,383285002240,Consumer Electronics,Technology,Consumer Electronics,161000,2.95T
1,MSFT,Microsoft Corporation,0.885,377.33,377.85,108530999296,0.49714,25.523,2803560087552,2770016403456,2.18,12.699986,105681002496,218310000640,Software - Infrastructure,Technology,Software - Infrastructure,221000,2.80T
2,GOOGL,Alphabet Inc.,1.05,138.03,138.49,96071000064,0.32333,17.1,1725866180608,1642851860480,1.24,6.297911,29446000640,297131999232,Internet Content & Information,Communication Services,Internet Content & Information,182381,1.73T
3,AMZN,"Amazon.com, Inc.",1.175,144.57,146.71,73978003456,0.13353,21.871,1507730522112,1618001657856,0.63,8.236888,166064996352,554027974656,Internet Retail,Consumer Cyclical,Internet Retail,1500000,1.51T
80,NVDA,NVIDIA Corporation,1.694,498.52,487.16,22161000448,0.49389,53.97,1191900741632,1196031213568,0.36,35.77366,11027000320,44870000640,Semiconductors,Technology,Semiconductors,26196,1.19T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LYFT,"Lyft, Inc.",1.789,10.27,10.29,-784977984,-0.18029,-4.456,4004568064,3497931008,-0.28,8.548657,1120690944,4353995776,Software - Application,Technology,Software - Application,4419,4.00B
79,AUR,"Aurora Innovation, Inc.",2.669,2.13,2.16,-815000000,0.0,-2.376,3305543168,1936743808,NaN,1.542553,125000000,2000000,Information Technology Services,Technology,Information Technology Services,1700,3.31B
49,TGLS,Tecnoglass Inc.,1.814,34.3,34.45,312592992,0.36785,5.357,1634437632,1674482176,9.23,3.181636,181148992,849779968,Building Materials,Basic Materials,Building Materials,8770,1.63B
53,BYND,"Beyond Meat, Inc.",2.249,6.6,6.6,-217475008,-0.62201,-6.552,435651072,1424972928,-0.18,NaN,1216547968,349635008,Packaged Foods,Consumer Defensive,Packaged Foods,787,435.65M


### 2.3 Getting data from yahoo_fin

In [220]:
#pip install yahoo_fin

In [221]:
from yahoo_fin.stock_info import get_data

In [222]:
ticker_list = ["amzn", "aapl", "meli","msft","crm"]

In [223]:
historical_data2 = {}
for ticker in ticker_list:
    historical_data2[ticker] = get_data(ticker,start_date="01/01/2018", end_date="23/11/2023", index_as_date = True, interval="1d")

/usr/local/lib/python3.10/dist-packages/yahoo_fin/stock_info.py:29: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  end_seconds = int(pd.Timestamp(end_date).timestamp())
/usr/local/lib/python3.10/dist-packages/yahoo_fin/stock_info.py:29: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  end_seconds = int(pd.Timestamp(end_date).timestamp())
/usr/local/lib/python3.10/dist-packages/yahoo_fin/stock_info.py:29: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  end_seconds = int(pd.Timestamp(end_date).timestamp())
/usr/local/lib/python3.10/dist-packages/yahoo_fin/stock_in

In [224]:
newdf = pd.DataFrame.from_dict(historical_data2["aapl"])
# Just the column thath I need
newdfh = newdf.filter(['close','ticker'], axis=1)
newdf = newdf.filter(['close'], axis=1)
# Renaming col
newdf.rename(columns={"close": "aapl"}, inplace=True)
newdf = newdf.reset_index()
newdf.rename(columns={"index": "date"}, inplace=True)


newdfh = newdfh.reset_index()
newdfh.rename(columns={"index": "date"}, inplace=True)

In [225]:
newdf2 = pd.DataFrame.from_dict(historical_data2["msft"])

newdfh2 = newdf2.filter(['close','ticker'], axis=1)
newdf2 = newdf2.filter(['close'], axis=1)

newdf2.rename(columns={"close": "msft"}, inplace=True)
newdf2 = newdf2.reset_index()
newdf2.rename(columns={"index": "date"}, inplace=True)


newdfh2 = newdfh2.reset_index()
newdfh2.rename(columns={"index": "date"}, inplace=True)

In [226]:
newdf3 = pd.DataFrame.from_dict(historical_data2["crm"])

newdfh3 = newdf3.filter(['close','ticker'], axis=1)
newdf3 = newdf3.filter(['close'], axis=1)

newdf3.rename(columns={"close": "crm"}, inplace=True)
newdf3 = newdf3.reset_index()
newdf3.rename(columns={"index": "date"}, inplace=True)

newdfh3 = newdfh3.reset_index()
newdfh3.rename(columns={"index": "date"}, inplace=True)

In [227]:
mergex1 = pd.merge(pd.merge(newdf,newdf2,how='outer',on='date'),newdf3,how='outer',on='date')
mergex1

,date,aapl,msft,crm
0,2018-01-02,43.064999,85.949997,104.410004
1,2018-01-03,43.057499,86.349998,105.290001
2,2018-01-04,43.257500,87.110001,106.680000
3,2018-01-05,43.750000,88.190002,108.099998
4,2018-01-08,43.587502,88.279999,108.860001
...,...,...,...,...
1479,2023-11-16,189.710007,376.170013,221.449997
1480,2023-11-17,189.690002,369.850006,221.220001
1481,2023-11-20,191.449997,377.440002,225.130005
1482,2023-11-21,190.639999,373.070007,224.320007


### 2.4 Getting data from OpenExchange API

In [228]:
df_exchange_rates = pd.read_json('https://openexchangerates.org/api/latest.json?app_id=XXXXXXXXXXXXXXXXX')
df_exchange_rates

,disclaimer,license,timestamp,base,rates
AED,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,3.672625
AFN,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,69.376917
ALL,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,93.718087
AMD,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,402.180000
ANG,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,1.801068
...,...,...,...,...,...
XPT,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,0.001093
YER,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,250.299958
ZAR,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,18.914405
ZMW,Usage subject to terms: https://openexchangera...,https://openexchangerates.org/license,2023-11-24 14:00:00,USD,23.402558


In [229]:
rows = ['EUR','BTC']
df_exchange_rates2 = df_exchange_rates.loc[rows,'rates']
df_exchange_rates2

EUR    0.915593
BTC    0.000026
Name: rates, dtype: float64

In [230]:
EUR = df_exchange_rates2[0]
EUR

0.915593

In [231]:
BTC = 1/df_exchange_rates2[1]
BTC

37796.570316531535

In [232]:
EUR_vig = {'USD / EUR':[EUR]}
EUR_Vigente = pd.DataFrame(data=EUR_vig)
EUR_Vigente

,USD / EUR
0,0.915593


In [233]:
BTC_vig = {'BTC':[BTC]}
BTC = pd.DataFrame(data=BTC_vig)
BTC

,BTC
0,37796.570317


In [234]:
df_concat_h2 = pd.concat([TRM_Vigente,EUR_Vigente, BTC,df_wti,df_brent_oil,df_gold], axis=1)
df_concat_h2

,TRM,USD / EUR,BTC,WTI Last Open,Brent Last Open,Gold Last Open
0,4092.33,0.915593,37796.570317,76.79,81.63,1991.2


In [235]:
format_mapping2={'TRM': '{:,.2f}', 'USD / EUR': '{:,.3f}', 'BTC': '{:,.2f}','WTI Last Open': '{:,.1f}', 'Brent Last Open': '{:,.1f}', 'Gold Last Open': '{:,.1f}'}

for key, value in format_mapping2.items():
    df_concat_h2[key] = df_concat_h2[key].apply(value.format)

df_concat_h2

,TRM,USD / EUR,BTC,WTI Last Open,Brent Last Open,Gold Last Open
0,"4,092.33",0.916,"37,796.57",76.8,81.6,"1,991.2"
